In [66]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain.vectorstores import FAISS
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [67]:

# Configure Google Generative AI
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise EnvironmentError("GOOGLE_API_KEY not found. Please set it in your .env file.")

genai.configure(api_key=GOOGLE_API_KEY)

# Initialize Language Model
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=GOOGLE_API_KEY)

# Load csv file

In [68]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='questions.csv',source_column='prompt')
dataset = loader.load()


# GoogleGenerativeAI for Word Embedding

In [69]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
e = embeddings.embed_query("What is your refund policy?")
type(e)
len(e)

768

# Create Vector DB

In [73]:
# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=dataset,
                                 embedding=embeddings)

In [78]:
# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.3)
docs = retriever.get_relevant_documents("duration of bootcamp?")
print(docs)

[Document(metadata={'source': 'What is the duration of this bootcamp? How long will it last?', 'row': 8}, page_content='prompt: What is the duration of this bootcamp? How long will it last?\nresponse: You can complete all courses in 3 months if you dedicate 2-3 hours per day.'), Document(metadata={'source': 'Can I attend this bootcamp while working full time?', 'row': 9}, page_content='prompt: Can I attend this bootcamp while working full time?\nresponse: Yes. This bootcamp is self-paced. You can learn on your own schedule.'), Document(metadata={'source': 'What if I don’t like this bootcamp?', 'row': 6}, page_content='prompt: What if I don’t like this bootcamp?\nresponse: As promised we will give you a 100% refund based on the guidelines (Please refer to our course refund policy before enrolling).'), Document(metadata={'source': 'What kind of skills can I expect to learn in this bootcamp?', 'row': 10}, page_content='prompt: What kind of skills can I expect to learn in this bootcamp?\nr

# Retrievel QA

In [79]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',retriever=retriever,input_key='query',return_source_documents=True)

# Q&A

In [81]:
print(chain("What if I don’t like this bootcamp?"))

{'query': 'What if I don’t like this bootcamp?', 'result': 'As promised, you will receive a 100% refund based on the guidelines.  Please refer to the course refund policy before enrolling.\n', 'source_documents': [Document(metadata={'source': 'What if I don’t like this bootcamp?', 'row': 6}, page_content='prompt: What if I don’t like this bootcamp?\nresponse: As promised we will give you a 100% refund based on the guidelines (Please refer to our course refund policy before enrolling).'), Document(metadata={'source': 'I’m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?', 'row': 4}, page_content='prompt: I’m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?\nresponse: We got you covered. Go ahead and watch our youtube videos if you like them and want to learn further then this bootcamp is the perfect extension.'), Document(metadata={'source': 'Can I attend this bootcamp while working full time?', 'row': 9}, page_

In [82]:
print(chain("""I have zero knowledge of Excel and belong to a non-technical
 background. Can I take this course?"""))

{'query': 'I have zero knowledge of Excel and belong to a non-technical\n background. Can I take this course?', 'result': 'Yes, this course is perfect for anyone with no prior Excel experience and a non-technical background.\n', 'source_documents': [Document(metadata={'source': 'I have zero knowledge of Excel and belong to a non-technical\n background. Can I take this course?', 'row': 16}, page_content='prompt: I have zero knowledge of Excel and belong to a non-technical\n background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never worked on excel and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.'), Document(metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}, page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nr